# Lab4 - st121413
Make Yolov4 run at inference time

Given the clue from other students (in Piazza), I made the following changes in order for my Yolov4 to be able to run.



## 1. Create Activator
There two activators currently in the configuration file that do not exist in the darknet.py. Namely, Mish and Linear.

Mish is given in the Piazza and Linear... I don't know what it is so I created a pass through Activator (y = x)

In [ ]:
class Mish(nn.Module):
     def __init__(self):
         super().__init__()

     def forward(self, x):
         return x * torch.tanh(F.softplus(x))

class LinearActivation(nn.Module):
     def __init__(self):
         super().__init__()

     def forward(self, x):
         return x

# When choosing activator

#Check the activation. 
#It is either Linear or a Leaky ReLU for YOLO
if activation == "leaky":
    activn = nn.LeakyReLU(0.1, inplace = True)
    module.add_module("leaky_{0}".format(index), activn)
elif activation == "mish":
    activn = Mish()
    module.add_module("mish_{0}".format(index), activn)
elif activation == "linear":
    activn = LinearActivation()
    module.add_module("linear_{0}".format(index), activn)
else:
    raise ValueError(f"No activation:{activation}")

## 2. Yolov4 configuration asked for 608
I searched and changed all 418 to 608. I should have done something better. 

## 3. Implement greater than 2 Route option
This took me a lot time to figure it out.

There two places that need to be adjust. The <code>previous_filter</code> in <code>darknet.py</code> and <code>myDarknet</code> class.

In [ ]:
''' From darknet.py '''

#If it is a route layer
elif (x["type"] == "route"):
    x["layers"] = x["layers"].split(',')
    #Start  of a route
    start = int(x["layers"][0])
    #end, if there exists one.
    try:
        end = int(x["layers"][1])
    except:
        end = 0
    #Positive anotation
    if start > 0: 
        start = start - index
    if end > 0:
        end = end - index
    route = EmptyLayer()
    module.add_module("route_{0}".format(index), route)
    if end < 0:
        filters = output_filters[index + start] + output_filters[index + end]
    else:
        filters= output_filters[index + start]
    if(len(x["layers"]) == 4): ######### <<<<<< I add this. Should have done better TT
        filters = 2048

In [ ]:
''' from myDarknet class '''
elif module_type == "route":
    # concat layers
    # print(module)
    # {'type': 'route', 'layers': ['-1', '-7']}
    layers = module["layers"]
    # layers = [int(a) for a in layers]
    
    #### Fix the preprocess #####
    for index, number in enumerate(layers):
        layers[index] = int(number)
        if(layers[index] > 0):
            layers[index] = layers[index] - i
    
    ##### Add concatenation of 4 layers and adjust for readability #####
    # print(layers)
    if len(layers) == 1:
        x = outputs[i + (layers[0])]
    elif len(layers) == 2:
        map1 = outputs[i + layers[0]]
        map2 = outputs[i + layers[1]]
        x = torch.cat((map1, map2), 1)
    elif len(layers) == 4:
        map1 = outputs[i + layers[0]]
        map2 = outputs[i + layers[1]]
        map3 = outputs[i + layers[2]]
        map4 = outputs[i + layers[3]]
        x = torch.cat((map1, map2, map3, map4), 1)
    else:
        raise ValueError(f"len of {layers} not defined")

## 4. Add 'maxpool' module

Since there is no maxpool module, we have to add it.

In [ ]:
''' From darknet.py '''

elif x["type"] == "maxpool":
    # {'type': 'maxpool', 'stride': '1', 'size': '5'}
    assert int(x['size']) % 2 ##### <<<<< This is from prof Matt and also the padding too.
    maxpool = nn.MaxPool2d(kernel_size=int(x['size']), stride=int(x['stride']), padding=int(x['size']) // 2)
    module.add_module("maxpool_{0}".format(index), maxpool)
else:
    print(x)
    raise ValueError(f"Module:{x['type']} not defined")  ##### <<<<< So that I know which module is missing

In [ ]:
''' From myDarknet class'''

###### for readability ######
if module_type in ["convolutional","upsample",'maxpool']:
    # do convolutional network
    x = self.module_list[i](x)

## 5. Change prep_image function in <code>util.py</code>

In [ ]:
''' From util.py '''

# def prep_image(img, inp_dim):
#     """
#     Prepare image for inputting to the neural network. 
    
#     Returns a Variable 
#     """
#     img = (letterbox_image(img, (inp_dim, inp_dim)))
#     img = img[:,:,::-1].transpose((2,0,1)).copy()
#     img = torch.from_numpy(img).float().div(255.0).unsqueeze(0)
#     return img


def prep_image(img, inp_dim):
    """
    Prepare image for inputting to the neural network. Returns a tensor.
    """
    # pylint: disable=no-member
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = letterbox_image(img, (inp_dim, inp_dim))
    return torch.from_numpy(img.transpose(2, 0, 1)).float().div(255.0).unsqueeze(0)


## Testing Result

![Drag Racing](des/det_dog-cycle-car.png)

In [ ]:
dog-cycle-car.png    predicted in  0.023 seconds
Objects Detected:    bicycle truck dog
----------------------------------------------------------
SUMMARY
----------------------------------------------------------
Task                     : Time Taken (in seconds)

Reading addresses        : 0.000
Loading batch            : 0.009
Detection (1 images)     : 0.092
Output Processing        : 0.000
Drawing Boxes            : 0.020
Average time_per_img     : 0.121
----------------------------------------------------------

I try some other images from coco validation set and here is the result

![Drag Racing](des/det_000000000139.jpg)
![Drag Racing](des/det_000000000285.jpg)
![Drag Racing](des/det_000000000632.jpg)
![Drag Racing](des/det_000000000724.jpg)
![Drag Racing](des/det_000000000776.jpg)
![Drag Racing](des/det_000000000785.jpg)
![Drag Racing](des/det_000000000802.jpg)
![Drag Racing](des/det_000000000872.jpg)
![Drag Racing](des/det_000000000885.jpg)
